In [ ]:
import numpy as np

### init random seeds
rd_seed = 42
np.random.seed(rd_seed)

from sklearn.pipeline import Pipeline
from pinard import preprocessor as pp

### Declare preprocessing pipeline components
preprocessing = [   ('id', pp.IdentityTransformer()),
                    ('savgol', pp.SavitzkyGolay()),
                    # ('derivate', pp.Derivate()), 
                    ('gaussian1', pp.Gaussian(order = 1, sigma = 2)),
                    ('gaussian2', pp.Gaussian(order = 2, sigma = 1)),
                    ('haar', pp.Wavelet('haar')),
                    ('savgol*savgol', Pipeline([('_sg1',pp.SavitzkyGolay()),('_sg2',pp.SavitzkyGolay())])),
                    ('gaussian1*savgol', Pipeline([('_g1',pp.Gaussian(order = 1, sigma = 2)),('_sg3',pp.SavitzkyGolay())])),
                    ('gaussian2*savgol', Pipeline([('_g2',pp.Gaussian(order = 1, sigma = 2)),('_sg4',pp.SavitzkyGolay())])),
                    ('haar*savgol', Pipeline([('_haar2',pp.Wavelet('haar')),('_sg5',pp.SavitzkyGolay())]))
                ]

In [ ]:
from sklearn.pipeline import FeatureUnion
from sklearn.preprocessing import MinMaxScaler
from sklearn.compose import TransformedTargetRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error, r2_score

In [ ]:
from pinard import nirs_set as n_set

### Load data
n = n_set.NIRS_Set('data')

### 1 set
# X, y = n.load('Xcal.csv', 'Ycal.csv', x_hdr=0, y_hdr=0, y_cols=0)
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = rd_seed)


## 2 sets
name = 'ArabifPlant_Growth_rate'
X_train, y_train = n.load(name + '_Xcal.csv', name + '_Ycal.csv', y_cols=0)
X_test, y_test = n.load(name + '_Xval.csv', name + '_Yval.csv', y_cols=0)


print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

In [ ]:
## Simple xgboost pipeline
from xgboost import XGBRegressor
from sklearn import svm


xgb =  XGBRegressor()
pipeline = Pipeline([
    ('scaler', MinMaxScaler()), 
    ('preprocessing', FeatureUnion(preprocessing)), 
    ('XGB', xgb)
    # ('SVM', svm.SVR())
])

estimator = TransformedTargetRegressor(regressor = pipeline, transformer = MinMaxScaler())

estimator.fit(X_train, y_train)
Y_preds = estimator.predict(X_test)
print("MAE", mean_absolute_error(y_test, Y_preds))
print("MSE", mean_squared_error(y_test, Y_preds))
print("MAPE", mean_absolute_percentage_error(y_test, Y_preds))
print("R²", r2_score(y_test, Y_preds))


In [ ]:
from sklearn.cross_decomposition import PLSRegression

## Simple PLS pipeline
pipeline = Pipeline([
    ('scaler', MinMaxScaler()), 
    ('preprocessing', FeatureUnion(preprocessing)), 
    ('pls', PLSRegression(n_components=7))
])

estimator = TransformedTargetRegressor(regressor = pipeline, transformer = MinMaxScaler())

estimator.fit(X_train, y_train)
Y_preds = estimator.predict(X_test)
print("MAE", mean_absolute_error(y_test, Y_preds))
print("MSE", mean_squared_error(y_test, Y_preds))
print("MAPE", mean_absolute_percentage_error(y_test, Y_preds))
print("R²", r2_score(y_test, Y_preds))


In [ ]:
## KERAS Model
##TODO > remove keraswrapper for sciKeras wrapper

from pinard.nirs_pipelines import FeatureAugmentation


import tensorflow
# from tensorflow.keras.wrappers.scikit_learn import KerasRegressor
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv1D, SpatialDropout1D,BatchNormalization,Flatten, Dropout, Input, MaxPool1D
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, CSVLogger, Callback
from tensorflow.keras import initializers

from typing import Dict, Iterable, Any
import scikeras
from scikeras.wrappers import KerasRegressor
import tensorflow_addons as tfa

import matplotlib.pyplot as plt

tensorflow.random.set_seed(rd_seed)
tensorflow.keras.backend.clear_session()

from sklearn.preprocessing import MinMaxScaler, RobustScaler, StandardScaler, QuantileTransformer

BATCH_SIZE = 50
EPOCHS = 4000
MIN_LR = 5e-5
MAX_LR = 1e-3
CYCLE_EPOCHS = 200
# initializer = initializers.RandomNormal(stddev=0.01)

steps_per_epoch = len(X_train) // BATCH_SIZE

clr = tfa.optimizers.CyclicalLearningRate(
    initial_learning_rate=MIN_LR,
    maximal_learning_rate=MAX_LR,
    scale_fn=lambda x: 1/(2.**(x-1)),
    step_size= CYCLE_EPOCHS * steps_per_epoch
)

class Auto_Save(Callback):
    best_weights = []
    def __init__(self, filepath):
        super(Auto_Save, self).__init__()
        self.best = np.Inf
        self.filepath = filepath
        
        
    def on_epoch_end(self, epoch, logs=None):
        current_loss = logs.get('val_loss')
        # print('\nLearning rate: {}\n'.format(self.model.optimizer.lr))
        print("Learning rate:", clr(self.model.optimizer.iterations.numpy()).numpy())
        if np.less(current_loss, self.best):
            self.best = current_loss            
            Auto_Save.best_weights = self.model.get_weights()
            # print('Saving weights validation loss= {0:6.4f}'.format(current_loss))

    def on_train_end(self, logs=None):
        # print('\nSaved best {0:6.4f} to {} \n'.format(self.best, self.filepath))
        print('\nSaved best {0:6.4f}\n'.format(self.best))
        

def keras_model(meta: Dict[str, Any]):

    
    step = np.arange(0, steps_per_epoch * EPOCHS)
    lr = clr(step)
    plt.plot(step, lr)
    plt.xlabel("Steps")
    plt.ylabel("Learning Rate")
    plt.show()


    input_shape = meta["X_shape_"][1:]
    model = Sequential()
    model.add(Input(shape=input_shape))
    model.add(SpatialDropout1D(0.2))
    model.add(Conv1D (filters=64, kernel_size=3, padding="same", activation='swish'))
    model.add(Conv1D (filters=64, kernel_size=3, padding="same", activation='swish'))
    model.add(MaxPool1D(pool_size=7,strides=5))
    model.add(SpatialDropout1D(0.2))
    model.add(Conv1D (filters=128, kernel_size=3, padding="same", activation='swish'))
    model.add(Conv1D (filters=128, kernel_size=3, padding="same", activation='swish'))
    model.add(MaxPool1D(pool_size=7,strides=5))
    model.add(SpatialDropout1D(0.2))
    model.add(Flatten())
    model.add(Dense(units=2048, activation="swish"))
    model.add(Dropout(0.2))
    model.add(Dense(units=2048, activation="swish"))
    model.add(Dense(units=1, activation="sigmoid"))
    model.compile(loss = 'mean_squared_error', metrics=['mse'], optimizer = Adam(clr))
    model.summary()
    return model

    
    
y_scaler = MinMaxScaler()

y_scaler.fit(y_train.reshape((-1,1)))
y_valid = y_scaler.transform(y_test.reshape((-1,1)))

transformer_pipe = Pipeline([
    ('scaler', MinMaxScaler()), 
    ('preprocessing', FeatureAugmentation(preprocessing)), 
])

transformer_pipe.fit(X_train)
X_valid = transformer_pipe.transform(X_test)
print(X_valid.shape)

model_filepath = 'model_test.hdf5'
earlyStopping = EarlyStopping(monitor='val_loss', patience=300, verbose=0, mode='min') 
# reduce_lr_loss = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=200, verbose=1, min_delta=0.001, mode='min')
# mcp_save = ModelCheckpoint(model_filepath, initial_value_threshold=0.008, save_best_only=True, monitor='val_loss', mode='min')#, save_freq=50) 
# logger = CSVLogger("log.csv", append=True)
auto_save = Auto_Save(model_filepath)

kregressor = KerasRegressor(model = keras_model,
                            callbacks=[earlyStopping, auto_save],
                            # callbacks=[reduce_lr_loss, changer],
                            epochs=EPOCHS, 
                            fit__batch_size=BATCH_SIZE,
                            # fit__validation_split=0.2,
                            fit__validation_data = (X_valid, y_valid),
                            verbose = 2)
# #  metrics=[KerasRegressor.r_squared])


pipeline = Pipeline([
    ('trans', transformer_pipe), 
    ('KerasNN', kregressor)
])


estimator = TransformedTargetRegressor(regressor = pipeline, transformer = y_scaler)
estimator.fit(X_train, y_train)

# # estimator.fit(X, y)
# estimator.regressor_[1].model_.load_weights(model_filepath)
estimator.regressor_[1].model_.set_weights(Auto_Save.best_weights)
Y_preds = estimator.predict(X_test)
print("MAE", mean_absolute_error(y_test, Y_preds))
print("MSE", mean_squared_error(y_test, Y_preds))
print("MAPE", mean_absolute_percentage_error(y_test, Y_preds))
print("R²", r2_score(y_test, Y_preds))

In [ ]:
from numba import cuda 
cuda.select_device(0) 
cuda.close() 

In [ ]:
def keras_model(meta: Dict[str, Any]):
    input_shape = meta["X_shape_"][1:]
    model = Sequential()
    model.add(Input(shape=input_shape))
    model.add(SpatialDropout1D(0.08))
    model.add(Conv1D (filters=8, kernel_size=15, strides=5, activation='selu'))
    model.add(Dropout(0.2))
    model.add(Conv1D (filters=64, kernel_size=21, strides=3, activation='relu'))
    model.add(BatchNormalization())
    model.add(Conv1D (filters=32, kernel_size=5, strides=3, activation='elu'))
    model.add(BatchNormalization())
    model.add(Flatten())
    model.add(Dense(16, activation='sigmoid'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss = 'mean_squared_error', optimizer = 'adam')
    model.summary()
    return model


In [ ]:
##Explainer

import shap

from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split
from sklearn.inspection import permutation_importance
from matplotlib import pyplot as plt



In [ ]:
# explain how the input to the 7th layer of the model explains the top two classes
# def map2layer(x, layer):
#     feed_dict = dict(zip([model.layers[0].input], [preprocess_input(x.copy())]))
#     return K.get_session().run(model.layers[layer].input, feed_dict)


# layer_in = estimator.regressor_[2].model_.layers[3]
# layer_out = estimator.regressor_[2].model_.layers[-1]
# layer_out

# e = shap.GradientExplainer((layer_in.input, layer_out.output), map2layer(preprocess_input(X.copy()), 7))

# kmodel = estimator.regressor_[2].model_
# explainer = shap.GradientExplainer(kmodel, X_train)

xtt = estimator.regressor_[:-1].transform(X_test)
print(X_test.shape, xtt.shape)
shap_values = explainer.shap_values(xtt[1])

In [ ]:
X_train_summary = shap.kmeans(X_train, 10)
explainer = shap.KernelExplainer(estimator.predict, X_train_summary)
shap_values = explainer.shap_values(X_test[0:5])


In [ ]:
shap.initjs()
shap.force_plot(explainer.expected_value, shap_values)

In [ ]:
#Global summary
shap.summary_plot(shap_values, X_test, plot_type="bar", max_display=25)

In [ ]:
# local summary
shap.summary_plot(shap_values, X_test[4])

In [ ]:
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (30,8)

X_ = np.arange(350, 350+shap_values[0].shape[0])
for s in shap_values:
    plt.plot(X_, s, alpha=0.3)
plt.title('SHAP values along the spectrum')
# plt.show()

for sx in  X_test[0:shap_values.shape[0]]:
    plt.plot(X_, sx)
plt.show()

In [ ]:
import shap

# X_train_summary = shap.kmeans(X_train, 10)

shap_values2 = explainer2.shap_values(X_test[0:5])

shap.plots.waterfall(shap_values2, max_display=100)


In [ ]:
explainer2 = shap.Explainer(estimator.predict, X_train[0:30])
shap_values2 = explainer2(X_test[0:10], max_evals=4400)


In [ ]:
shap.plots.waterfall(shap_values2[2], shap_values2[0][0], X_test[0])


In [ ]:


import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (10,5)
print(shap_values[0].shape)

X_ = np.arange(0, shap_values[0].shape[0])
for s in X_test:
    plt.plot(X_, s)
plt.title('SHAP values along the spectrum')
plt.show()